In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Load Data

In [49]:
# No rescaling needed. MNIST data is in range [0, 1]
mnist = input_data.read_data_sets("tf_data/")

Extracting tf_data/train-images-idx3-ubyte.gz
Extracting tf_data/train-labels-idx1-ubyte.gz
Extracting tf_data/t10k-images-idx3-ubyte.gz
Extracting tf_data/t10k-labels-idx1-ubyte.gz


## Model

In [73]:
# Generator Neural Network
class Generator(object):
    def __init__(self):
        """ Init the model with hyper-parameters etc. """
        with tf.variable_scope("G"):
            # Layer 1 Params
            self.weights1 = tf.Variable(tf.zeros([784, 1024]))
            self.biases1 = tf.Variable(tf.zeros([1024]))
            # Layer 2 Params
            self.weights2 = tf.Variable(tf.zeros([1024, 512]))
            self.biases2 = tf.Variable(tf.zeros([512]))
            # Layer 3 Params
            self.weights3 = tf.Variable(tf.zeros([512, 256]))
            self.biases3 = tf.Variable(tf.zeros([256]))
            # Store params in list
            self.param_list = [self.weights1, self.biases1, self.weights2, self.biases2, 
                               self.weights3, self.biases3]
            
    def inference(self, x):
        """ This is the forward calculation from x to y """
        layer_1 = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, weights1) + biases1), .3)
        layer_2 = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, weights2) + biases2), .3)
        layer_3 = tf.nn.dropout(tf.nn.leaky_relu(tf.matmul(x, weights3) + biases3), .3)
        return tf.tanh(layer3)
    
    def loss(self, batch_x, discriminator):
        y_predict = discriminator.inference(self.inference(batch_x))
        self.loss = tf.losses.log_loss(y, y_predict, name="loss")

## Train

In [50]:
num_epochs = 100

for _ in range(num_epochs):
    real_images, _ = mnist.train.next_batch(100)
#    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})